# Road Following Debug 

In this notebook, we will debug and adjust the steering settings to allow the jetBot to smoothly run on our track. 

#### This Notebook is largely based on the "live_demo" notebook that is found in the Sparkfun notebook for road following. It is meant to be a code to help debug your robot/code.
#### Modified by the Skonk Works robotics team for the STARR program 02/22/2021

### IMPORTANT!! 
#### The following code will disable your LED printout showing the IP address of your robot. You MUST run the last cell in this notebook to reenable your LED operations. If something happens during the operation of this notebook, simply reboot your robot, return to this notebook and run the last cell to reenable the functionality of your LED

### DO NOT run the following two cells again after rebooting!

In [1]:
import os
import getpass

#### The following lines of code will cause your robot to reboot. You will need to type in your password into the text widget.

In [ ]:
password = getpass.getpass()
os.system('echo %s | sudo -S mv  /usr/local/lib/python3.6/dist-packages/jetbot-0.4.0-py3.6.egg/jetbot/apps/stats.py /usr/local/lib/python3.6/dist-packages/jetbot-0.4.0-py3.6.egg/jetbot/apps/stats.py.orig ' % (password))
os.system('echo %s | sudo -S reboot ' % (password))

 ······


### Load Trained Model

We will assume that you have already downloaded ``best_steering_model_xy.pth`` to work station as instructed in "train_model.ipynb" notebook. Now, you should upload model file to JetBot in to this notebooks's directory. Once that's finished there should be a file named ``best_steering_model_xy.pth`` in this notebook's directory.

Execute the code below to initialize the PyTorch model. This should look very familiar from the training notebook.

> Please make sure the file has uploaded fully before calling the next cell

In [1]:
import torchvision
import torch

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

Next, load the trained weights from the ``best_steering_model_xy.pth`` file that you uploaded.

In [2]:
model.load_state_dict(torch.load('best_maze_model_xy_251.pth'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

In [3]:
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

### Creating the Pre-Processing Function

We have now loaded our model, but there's a slight issue. The format that we trained our model doesnt exactly match the format of the camera. To do that, we need to do some preprocessing. This involves the following steps:

1. Convert from HWC layout to CHW layout
2. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
3. Transfer the data from CPU memory to GPU memory
4. Add a batch dimension

In [4]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Awesome! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

The command to display your camera to your notebook is currently commented out so the robot will run more efficiently.
If you wish to have the camera display to your notebook uncomment the display command.

In [5]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera()

image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

#display(image_widget)

We'll also create our robot instance which we'll need to drive the motors.

In [6]:
from jetbot import Robot

robot = Robot()

In [ ]:
robot.stop()

#### Now create LED functions to show which direction our robot is steering. These will be used to test how the robot is interpreting it's scene while remaining stationary.

In [7]:
import qwiic_micro_oled
cyclops = qwiic_micro_oled.QwiicMicroOled()

In [8]:
def arrow_right():
    cyclops.begin()
    cyclops.clear(cyclops.PAGE)

    cyclops.pixel(62,24)
    cyclops.line_v(61,23,3)
    cyclops.line_v(60,22,5)
    cyclops.line_v(59,21,7)
    cyclops.line_v(58,20,9)
    cyclops.line_v(57,19,11)
    cyclops.line_v(56,18,13)
    cyclops.line_v(55,17,15)
    cyclops.line_v(54,16,17)
    cyclops.line_v(53,15,19)
    cyclops.line_v(52,14,21)
    cyclops.line_v(51,13,23)
    cyclops.line_v(50,12,25)
    cyclops.line_v(49,11,27)
    cyclops.line_v(48,10,29)
    cyclops.line_v(47,9,31)
    cyclops.line_v(46,8,33)
    cyclops.line_v(45,7,35)
    cyclops.line_v(44,6,37)

    cyclops.rect_fill(3,16,41,16)

    cyclops.display()

def arrow_left():
    cyclops.begin()
    cyclops.clear(cyclops.PAGE)

    cyclops.pixel(2,24)
    cyclops.line_v(3,23,3)
    cyclops.line_v(4,22,5)
    cyclops.line_v(5,21,7)
    cyclops.line_v(6,20,9)
    cyclops.line_v(7,19,11)
    cyclops.line_v(8,18,13)
    cyclops.line_v(9,17,15)
    cyclops.line_v(10,16,17)
    cyclops.line_v(11,15,19)
    cyclops.line_v(12,14,21)
    cyclops.line_v(13,13,23)
    cyclops.line_v(14,12,25)
    cyclops.line_v(15,11,27)
    cyclops.line_v(16,10,29)
    cyclops.line_v(17,9,31)
    cyclops.line_v(18,8,33)
    cyclops.line_v(19,7,35)
    cyclops.line_v(20,6,37)

    cyclops.rect_fill(20,16,41,16)

    cyclops.display()
    
def arrow_straight():
    cyclops.begin()
    cyclops.clear(cyclops.PAGE)

    cyclops.pixel(62,24)
    cyclops.line_v(61,23,3)
    cyclops.line_v(60,22,5)
    cyclops.line_v(59,21,7)
    cyclops.line_v(58,20,9)
    cyclops.line_v(57,19,11)
    cyclops.line_v(56,18,13)
    cyclops.line_v(55,17,15)
    cyclops.line_v(54,16,17)
    cyclops.line_v(53,15,19)
    cyclops.line_v(52,14,21)
    cyclops.line_v(51,13,23)
    cyclops.line_v(50,12,25)
    cyclops.line_v(49,11,27)
    cyclops.line_v(48,10,29)
    cyclops.line_v(47,9,31)
    cyclops.line_v(46,8,33)
    cyclops.line_v(45,7,35)
    cyclops.line_v(44,6,37)
    
    cyclops.pixel(2,24)
    cyclops.line_v(3,23,3)
    cyclops.line_v(4,22,5)
    cyclops.line_v(5,21,7)
    cyclops.line_v(6,20,9)
    cyclops.line_v(7,19,11)
    cyclops.line_v(8,18,13)
    cyclops.line_v(9,17,15)
    cyclops.line_v(10,16,17)
    cyclops.line_v(11,15,19)
    cyclops.line_v(12,14,21)
    cyclops.line_v(13,13,23)
    cyclops.line_v(14,12,25)
    cyclops.line_v(15,11,27)
    cyclops.line_v(16,10,29)
    cyclops.line_v(17,9,31)
    cyclops.line_v(18,8,33)
    cyclops.line_v(19,7,35)
    cyclops.line_v(20,6,37)
    
    cyclops.rect_fill(20,16,24,16)
    
    cyclops.display()



Now, we will define sliders to control JetBot
> Note: We have initialize the slider values for best known configurations, however these might not work for your dataset, therefore please increase or decrease the sliders according to your setup and environment

1. Speed Control (speed_gain_slider): To start your JetBot increase ``speed_gain_slider`` 
2. Steering Gain Control (steering_gain_sloder): If you see JetBot is woblling, you need to reduce ``steering_gain_slider`` till it is smooth
3. Steering Bias control (steering_bias_slider): If you see JetBot is biased towards extreme right or extreme left side of the track, you should control this slider till JetBot start following line or track in the center.  This accounts for motor biases as well as camera offsets

> Note: You should play around above mentioned sliders with lower speed to get smooth JetBot road following behavior.

In [9]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.2, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

Next, let's display some sliders that will let us see what JetBot is thinking.  The x and y sliders will display the predicted x, y values.

The steering slider will display our estimated steering value.  Please remember, this value isn't the actual angle of the target, but simply a value that is
nearly proportional.  When the actual angle is ``0``, this will be zero, and it will increase / decrease with the actual angle.  

In [10]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

Next, we'll create a function that will get called whenever the camera's value changes. This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. Compute the approximate steering value
4. Control the motors using proportional / derivative control (PD)

In [11]:
angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last
    image = change['new']
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    test_val_left  = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    test_val_right = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    
    if abs(test_val_right-test_val_left) < 0.20 :
        arrow_straight()
    elif test_val_right > test_val_left :
        arrow_left()
    else :
        arrow_right()
    
execute({'new': camera.value})

Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing.

We accomplish that with the observe function.

In [12]:
camera.observe(execute, names='value')

Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame. 

You can now place JetBot on  Lego or Track you have collected data on and see whether it can follow track.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [13]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

### IMPORTANT!! The following cell must be exectuted in order to return functionality of your LED screen. It should not be necessary to reboot your robot.

In [14]:
import getpass
import os
password = getpass.getpass()
os.system('echo %s | sudo -S mv  /usr/local/lib/python3.6/dist-packages/jetbot-0.4.0-py3.6.egg/jetbot/apps/stats.py.orig /usr/local/lib/python3.6/dist-packages/jetbot-0.4.0-py3.6.egg/jetbot/apps/stats.py ' % (password))

 ······


0

### Conclusion
That's it for this debugging demo! 